In [14]:
import os
import pandas as pd
import requests as re
import json

In [3]:
# Get the neighborhood data

nyc = pd.read_csv('../data/data/nyc_geo_data_cleaned.csv', index_col=0)

In [4]:
nyc

,id,borough,neighborhood,longitude,latitude
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834
...,...,...,...,...,...
301,nyu_2451_34572.302,Manhattan,Hudson Yards,-74.000111,40.756658
302,nyu_2451_34572.303,Queens,Hammels,-73.805530,40.587338
303,nyu_2451_34572.304,Queens,Bayswater,-73.765968,40.611322
304,nyu_2451_34572.305,Queens,Queensbridge,-73.945631,40.756091


In [5]:
## define yelp search function

def yelpSearch(latitude, longitude, query, radius):
    """
    Accepts search term string, lat & lon as floats, radius as int
    """

    yelp_key = os.environ["YELP_API_KEY"]

    yelp_headers = {
        "Accept": "application/json",
        "Authorization": "Bearer " + yelp_key
    }
    
    yelp_url = "https://api.yelp.com/v3/businesses/search"

    yelp_params = {
        "term": query,
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "limit": 50
    }

    response = re.get(yelp_url, params=yelp_params, headers=yelp_headers)

    return response

In [10]:
response = yelpSearch(str(40.894705), str(-73.847201), 'restaurant', 500)
response.json()['businesses'][0]['categories'][0]['alias']

'caribbean'

In [13]:
# dict for storage
yelp_dict = {}

for row in nyc.itertuples():
    print(row.id)
    # call the API
    response = yelpSearch(str(row.latitude), str(row.longitude), 'restaurant', 500)

    # set up lists
    yelp_categories = []

    # parse the category data
    for business in response.json()['businesses']:
        for category in business['categories']:
            yelp_categories.append(category['alias'])

    yelp_dict[row.id] = {
        'yelp_categories': yelp_categories
    }

nyu_2451_34572.1
nyu_2451_34572.2
nyu_2451_34572.3
nyu_2451_34572.4
nyu_2451_34572.5
nyu_2451_34572.6
nyu_2451_34572.7
nyu_2451_34572.8
nyu_2451_34572.9
nyu_2451_34572.10
nyu_2451_34572.11
nyu_2451_34572.12
nyu_2451_34572.13
nyu_2451_34572.14
nyu_2451_34572.15
nyu_2451_34572.16
nyu_2451_34572.17
nyu_2451_34572.18
nyu_2451_34572.19
nyu_2451_34572.20
nyu_2451_34572.21
nyu_2451_34572.22
nyu_2451_34572.23
nyu_2451_34572.24
nyu_2451_34572.25
nyu_2451_34572.26
nyu_2451_34572.27
nyu_2451_34572.28
nyu_2451_34572.29
nyu_2451_34572.30
nyu_2451_34572.31
nyu_2451_34572.32
nyu_2451_34572.33
nyu_2451_34572.34
nyu_2451_34572.35
nyu_2451_34572.36
nyu_2451_34572.37
nyu_2451_34572.38
nyu_2451_34572.39
nyu_2451_34572.40
nyu_2451_34572.41
nyu_2451_34572.42
nyu_2451_34572.43
nyu_2451_34572.44
nyu_2451_34572.45
nyu_2451_34572.46
nyu_2451_34572.47
nyu_2451_34572.48
nyu_2451_34572.49
nyu_2451_34572.50
nyu_2451_34572.51
nyu_2451_34572.52
nyu_2451_34572.53
nyu_2451_34572.54
nyu_2451_34572.55
nyu_2451_34572.56
n

In [16]:
yelp_dict

{'nyu_2451_34572.1': {'yelp_categories': ['caribbean',
   'breakfast_brunch',
   'caribbean',
   'pizza',
   'breakfast_brunch',
   'burgers',
   'sandwiches',
   'caribbean',
   'comfortfood',
   'soulfood',
   'caribbean',
   'pizza',
   'grocery',
   'cafes',
   'sandwiches',
   'sandwiches']},
 'nyu_2451_34572.2': {'yelp_categories': ['chinese',
   'delis',
   'sandwiches',
   'chinese',
   'delis',
   'pizza',
   'coffee',
   'delis',
   'burgers',
   'hotdogs',
   'lounges',
   'restaurants',
   'hotdogs',
   'pizza']},
 'nyu_2451_34572.3': {'yelp_categories': ['caribbean',
   'seafood',
   'bars',
   'breakfast_brunch',
   'greek',
   'chinese',
   'seafood',
   'tradamerican',
   'breakfast_brunch',
   'diners',
   'pizza',
   'caribbean',
   'tradamerican',
   'caribbean',
   'pizza',
   'italian',
   'newamerican',
   'comfortfood',
   'danceclubs',
   'cafes',
   'hotdogs',
   'burgers',
   'coffee']},
 'nyu_2451_34572.4': {'yelp_categories': []},
 'nyu_2451_34572.5': {'yelp

In [17]:
with open('yelp_restaurant_categories.json', 'w') as fp:
    json.dump(yelp_dict, fp)